In [46]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_validate,train_test_split
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


In [47]:
# For comparing our model scores
modelScores = {}

In [57]:
space_df = pd.read_csv('data/train.csv')
space_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [65]:
# Split the dataset into training and testing data
train_df, test_df = train_test_split(space_df, test_size=.2, random_state=123)
X_train, y_train = train_df.drop(columns=['Transported']), train_df[['Transported']]
X_test, y_test = test_df.drop(columns=['Transported']), test_df[['Transported']]

In [66]:
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
%matplotlib inline

In [67]:
spaceTrain = pd.read_csv('data/train.csv')
spaceTest = pd.read_csv('data/test.csv')

In [68]:
spaceTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [69]:
# Let's deal with the slashes in the 'cabin' column first:
def cabinUpdate(df):
    # Split Cabin columns string on the slash and rename its columns
    cabinNew = df.Cabin.str.split('/', expand=True)
    cabinNew.columns = ['Cabin-1', 'Cabin-2', 'Cabin-3']
    # Copy the input dataframe to avoid mutating the original
    df1 = df.copy()
    # Add expanded columns to our copied dataframe:
    for col in cabinNew:
        df1[col] = cabinNew[col]
        print(cabinNew[col].value_counts())
    # Reorder the columns in the new dataframe:
    cols = df1.columns.tolist()
    cols = cols[:3] + cols[-3:] + cols[4:-3]
    df1 = df1[cols]
    return df1
# Update our DF's structure based on the Cabin column
spaceTrain = cabinUpdate(spaceTrain).copy()
# Split data into train and test
train_df, test_df = train_test_split(spaceTrain, test_size=.2, random_state=123)
X_train, y_train = train_df.drop(columns=['Transported']), train_df['Transported']
X_test, y_test = test_df.drop(columns=['Transported']), test_df['Transported']
X_train.head()

F    2794
G    2559
E     876
B     779
C     747
D     478
A     256
T       5
Name: Cabin-1, dtype: int64
82      28
86      22
19      22
56      21
176     21
        ..
1644     1
1515     1
1639     1
1277     1
1894     1
Name: Cabin-2, Length: 1817, dtype: int64
S    4288
P    4206
Name: Cabin-3, dtype: int64


,PassengerId,HomePlanet,CryoSleep,Cabin-1,Cabin-2,Cabin-3,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
7074,7527_01,Earth,False,F,1561,P,TRAPPIST-1e,18.0,False,0.0,732.0,2.0,13.0,47.0,Elle Flowensley
6710,7083_01,Europa,True,C,259,S,TRAPPIST-1e,32.0,False,0.0,0.0,0.0,0.0,0.0,Betenar Pirejus
2569,2755_01,Europa,True,B,85,P,55 Cancri e,36.0,False,0.0,0.0,0.0,0.0,0.0,Beneba Glousspidy
3850,4127_01,Europa,NaN,B,130,P,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,NaN,Dsch Ainserfle
7450,7972_02,Europa,False,B,260,P,TRAPPIST-1e,35.0,False,NaN,246.0,25.0,42.0,3730.0,Atinon Cattyried


In [70]:
# Define our categories of columns:
cat = ['HomePlanet',  'Cabin-1', 'Cabin-3', 'Destination']
num = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin-2']
binary = ['CryoSleep', 'VIP']
drop = ['Name']
target = 'Transported'
# Make the column transformer:
preprocessor = make_column_transformer(
        # Apply standard scaling to all our numeric features
        (make_pipeline(SimpleImputer(strategy='mean'), StandardScaler()), num),
        # Make a pipeline for our categorical features
        # 1. First fill NaN's with most frequent of each feature
        # 2. Then use one hot encoding on result
        (make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore')), cat),
        # One-hot encoding of our binary column, 'sex,' so we aren't wasting space
        (make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(drop="if_binary", dtype=int,handle_unknown='ignore')), binary),
        ('drop', drop)
)

In [34]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation
    
    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [35]:

# Create the random grid
random_grid = {'randomforestclassifier__n_estimators': [90,100,115,130],
               'randomforestclassifier__criterion': ['gini','entropy'],
               'randomforestclassifier__max_depth': range(2,20,1),
               'randomforestclassifier__min_samples_split': range(2,10,1),
               'randomforestclassifier__min_samples_leaf': range(1,10,1),
               'randomforestclassifier__max_features': ['log2','auto']}

In [36]:
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [37]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
pipe_rf = make_pipeline(preprocessor,RandomForestClassifier())
#Pipeline(steps = ['rf',RandomForestRegressor()])
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(pipe_rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer()),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['Age',
                                                                                'RoomService',
                                                                                'FoodCourt',
                                                                                'S

In [38]:
rf_random.best_params_

{'randomforestclassifier__n_estimators': 130,
 'randomforestclassifier__min_samples_split': 9,
 'randomforestclassifier__min_samples_leaf': 4,
 'randomforestclassifier__max_features': 'log2',
 'randomforestclassifier__max_depth': 11,
 'randomforestclassifier__criterion': 'gini'}

In [39]:
rf_pipeline = make_pipeline(preprocessor,RandomForestClassifier(n_estimators=130,min_samples_split=9,min_samples_leaf=4,max_features='log2',max_depth=11,criterion='gini'))

In [40]:
rf_pipeline.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck',
                                                   'Cabin-2']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
              

In [42]:
rf_pipeline.score(X_test,y_test)

0.81196089706728

In [71]:
X_train, y_train = spaceTrain.drop(columns=['Transported']), spaceTrain[['Transported']]
rf_pipeline.fit(X_train, y_train)
# Manual transformation
spaceTest = cabinUpdate(spaceTest).copy()
# Make our predictions:
spaceshipPredictions = rf_pipeline.predict(spaceTest)
mean_std_cross_val_scores(rf_pipeline,X_train,y_train,return_train_score = True)

C:\Users\jmbaer\anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


F    1445
G    1222
E     447
B     362
C     355
D     242
A      98
T       6
Name: Cabin-1, dtype: int64
4       21
31      18
197     16
294     16
228     14
        ..
1170     1
904      1
1174     1
356      1
1503     1
Name: Cabin-2, Length: 1505, dtype: int64
S    2093
P    2084
Name: Cabin-3, dtype: int64


C:\Users\jmbaer\anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\jmbaer\anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\jmbaer\anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\jmbaer\anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. 

fit_time       0.545 (+/- 0.013)
score_time     0.039 (+/- 0.001)
test_score     0.799 (+/- 0.017)
train_score    0.853 (+/- 0.004)
dtype: object

In [55]:
result = zip(spaceTest['PassengerId'], pd.Series(spaceshipPredictions))
out = pd.DataFrame(result, columns=['PassengerId', 'Transported']).sort_values('PassengerId')
out.to_csv('./results/rf_hyper.csv', index=False)
out.describe()

,PassengerId,Transported
count,4277,4277
unique,4277,2
top,0013_01,True
freq,1,2143
